# Algoritmos de optimización - Trabajo práctico<br>
Nombre y Apellidos: Julen Cortázar Noguerol<br>
Url: https://github.com/jcortazar073/03MIAR---Algoritmos-de-Optimizacion/blob/main/TP_Julen_Cortazar_Noguerol.ipynb<br>
Problema: 3. Combinar cifras y operaciones

Descripción del problema:

La actividad consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad
dada. Un ejemplo sería para obtener el 4:

$4+2-6/3*1=4$

Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:
- ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
- ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo?

----------

En primera instancia conviene definir listas con los elementos disponibles. Dado que no se permite repetición, puede ser útil emplear el método 'pop()' de Python para eliminar los elementos ya usados, ya que su complejidad computacional es O(1). Definimos por tanto las listas:

In [2]:
cifras = [str(x) for x in range(1,10)]      # Definimos la lista de dígitos del 1 al 9
operadores = ['+', '-', '/', '*']           # Definimos la lista de operadores disponibles

**1. ¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?**

Teniendo en cuenta todas las restricciones, puede calcularse el número de posibilidades como el producto de las permutaciones sin repetición de las 9 cifras en los 5 posibles 'huecos' para cada una, por las permutaciones sin repetición de los cuatro operadores en las cuatro posibles posiciones. Si llamamos $N$ al número total de posibilidades:

$N = [9!/(9-5)!]*4! = 362880$

Esto significa que se pueden obtener un total de 362880 expresiones diferentes, pero no necesariamente que se vayan a obtener resultados diferentes en cada una de las expresiones.

Pero esta cantidad de posibilidades puede reducirse de forma considerable teniendo en cuenta la jerarquía de operaciones y la propiedad conmutativa de la suma, resta y multiplicación. En este sentido, la operación más restrictiva es la división. Si nos limitamos a resultados enteros, la división sólo puede situarse entre numeros divisibles, es decir:

- 8 combinaciones posibles dividiendo entre 1.
- 3 combinaciones posibles dividiendo entre 2.
- 2 combinaciones posibles dividiendo entre 3.
- 1 combinación posible dividiendo entre 4.

Lo que da un total de 14 combinaciones posibles para el operador división. A partir de aquí, tendríamos la permutación sin repetición de 8 números en 4 posiciones, siendo uno de los números el resultado de la división, y la permutación sin repetición de 3 operadores en 3 posiciones, todo ello multiplicado por las 15 combinaciones posibles del operador división. De esta forma:

$N = [8!/(8-4)!]*3!*14 = 141120$

Lo que reduce el cálculo a menos de la mitad de posibilidades. Se podría argumentar que con este enfoque se pierde generalidad, ya que, por ejemplo, $1/2*4$ da un resultado entero. Sin embargo, aprovechando la propiedad conmutativa del producto, se puede demostrar que el resultado de esa operación ya está contemplada en el cálculo anterior. Suponiendo tres enteros $a$, $b$ y $c$ tal que $c = n*b$ siendo $n$ un número entero también, aprovechando la propiedad conmutativa:

$a/b*c = a/b*n*b = n*a =n*b/b*a = c/b*a$

Por tanto queda demostrado que el segundo cálculo de posibilidades abarca cualquier resultado entero posible.

In [3]:
import numpy as np

N1 = (np.math.factorial(4)*np.math.factorial(9))/np.math.factorial(9-5)

N2 = (14*np.math.factorial(3)*np.math.factorial(8))/np.math.factorial(8-4)

print(f'N1 = {int(N1)}\nN2 = {int(N2)}')

N1 = 362880
N2 = 141120


---

**2. Modelo para el espacio de soluciones.**
**¿Cuál es la estructura de datos que mejor se adapta al problema? Arguméntalo.**


En el caso del problema propuesto la estructura de datos más representativa y completa sería un diccionario. Dado que las expresiones a evaluar son únicas, podrían utilizarse de clave, siendo el valor asociado a la clave el resultado de la operación. Por otro lado, la evaluación de las expresiones es inmediata, por lo que una lista de todas las expresiones a evaluar sería suficiente.

En el siguiente código se muestra un ejemplo de ambas configuraciones:

In [4]:
sol_dict = {'2+3-8/4*5': eval('2+3-8/4*5'), '2-3+8/4*5': eval('2-3+8/4*5')}     # Posible representación del espacio de soluciones con diccionarios
sol_list = ['2+3-8/4*5', '2-3+8/4*5']                                           # Posible representación del espacio de soluciones con listas

print(f'Diccionario de soluciones: {sol_dict}\n')
print(f'Lista de expresiones: {sol_list}\n' +
      f'Lista de soluciones: {[eval(x) for x in sol_list]}')

Diccionario de soluciones: {'2+3-8/4*5': -5.0, '2-3+8/4*5': 9.0}

Lista de expresiones: ['2+3-8/4*5', '2-3+8/4*5']
Lista de soluciones: [-5.0, 9.0]


----------

**Según el modelo para el espacio de soluciones:**<br>
- **¿Cual es la función objetivo?**
- **¿Es un problema de maximización o minimización?**

Este problema no tiene una función objetivo como tal. Siendo rigurosos, podríamos tomar la propia definición del problema como función objetivo:

$f(a,b,c,d,e,\alpha, \beta, \gamma, \delta) = a(\alpha)b(\beta)c(\gamma)d(\delta)e$,<br> 
**tal que**  $a,b,c,d,e \in [1,9]$  **con**  $a\neq b\neq c\neq d\neq e$  **y**  $\alpha, \beta, \gamma, \delta\in [+,-,/,*]$  **con**  $\alpha\neq\beta\neq\gamma\neq\delta$.

En cuanto a máximización o minimización, en este problema se pide buscar tanto el mínimo como el máximo de la función descrita.

-----------

**Resolución del problema por fuerza bruta**

Para resolver el problema por fuerza bruta, se pueden calcular todas las permutaciones posibles de números y símbolos, evaluar las expresiones, y buscar el mínimo y el máximo. Para ello se va a utilizar la función `permutations` de la librería `itertools`:



In [5]:
from itertools import permutations
import numpy as np

cifras = [str(x) for x in range(1,10)]      # Definimos la lista de dígitos del 1 al 9
operadores = ['+', '-', '/', '*']           # Definimos la lista de operadores disponibles

# Construimos las permutaciones de ambos conjuntos de datos
cifras_perm = list(permutations(cifras, len(operadores)+1))
operadores_perm = list(permutations(operadores))

sol_dict = {} # Declaramos el espacio de soluciones

# Combinamos ambos conjuntos de datos
for set_cifras in cifras_perm:
  for set_operadores in operadores_perm:
    expresion =''
    cifras_lst = list(set_cifras)
    operadores_lst = list(set_operadores)
    while len(operadores_lst) > 0:
      expresion += cifras_lst.pop() + operadores_lst.pop()

    expresion+=cifras_lst.pop()
    sol_dict[eval(expresion)] = expresion

# Análisis de los resultados:
total_expresiones_únicas = len(sol_dict)

for x in list(sol_dict.keys()):
  if not x.is_integer():
    sol_dict.pop(x)

valores = [int(x) for x in list(sol_dict.keys())]

print(f'Se han encontrado un total de {len(cifras_perm)*len(operadores_perm)} expresiones posibles sin aplicar ninguna restricción,\n'+
      f'El resultado máximo de estas expresiones es {max(valores)} y el mínimo {min(valores)}, con un total de {len(valores)} resultados únicos.')

Se han encontrado un total de 362880 expresiones posibles sin aplicar ninguna restricción,
El resultado máximo de estas expresiones es 77 y el mínimo -69, con un total de 147 resultados únicos.


Podemos encapsular el código anterior en una función, generalizada para cualquier set de operadores y cifras, que devuelva una combinación de dígitos y elementos para un resultado dado:

In [6]:
def permutacion_operaciones_fuerza_bruta(cifras, operadores, resultado, sol_dict_input = None):
  from itertools import permutations
  # Bloque de código para ahorrar tiempo en búsquedas con el mismo set de
  # cifras y operaciones.
  if sol_dict_input !=None:
    try:
      return sol_dict_input[resultado]
    except:
      print('Resultado no encontrado en el set de soluciones proporcionado')
      return

  cifras_perm = list(permutations(cifras, len(operadores)+1))
  operadores_perm = list(permutations(operadores))
  
  sol_dict = {} # Declaramos el espacio de soluciones

  # Combinamos ambos conjuntos de datos
  for set_cifras in cifras_perm:
    for set_operadores in operadores_perm:
      expresion =''
      cifras_lst = list(set_cifras)
      operadores_lst = list(set_operadores)
      while len(operadores_lst) > 0:
        expresion += cifras_lst.pop() + operadores_lst.pop()

      expresion+=cifras_lst.pop()
      x = eval(expresion)
      if x.is_integer():
        sol_dict[int(x)] = expresion

  try:
    return sol_dict[resultado], sol_dict
  except:
    print('No se ha encontrado el resultado')
    return '', sol_dict

Pruebas de uso:

In [7]:
cifras = [str(x) for x in range(1,10)]      # Definimos la lista de dígitos del 1 al 9
operadores = ['+', '-', '/', '*']           # Definimos la lista de operadores disponibles

[expresion, sol_dict] = permutacion_operaciones_fuerza_bruta(cifras, operadores, 4)
print(f'{expresion} = {4}\n')

# Una vez se obtiene el diccionario de soluciones el cálculo es inmediato:
for i in range(-69, 78):
  expresion = permutacion_operaciones_fuerza_bruta(cifras, operadores, i, sol_dict)
  print(f'{expresion} = {i}\n')

2/4*6-8+9 = 4

1+6/3-8*9 = -69

2+6/3-8*9 = -68

4/2+3-8*9 = -67

6/3+4-8*9 = -66

6/3+5-8*9 = -65

4/2+6-8*9 = -64

6/3+7-8*9 = -63

6/2+7-8*9 = -62

4/1+7-8*9 = -61

5/1+7-8*9 = -60

6/1+7-8*9 = -59

3+8/4-7*9 = -58

6/3+4-7*9 = -57

8/4+5-7*9 = -56

8/4+6-7*9 = -55

3/1+6-7*9 = -54

6/3+8-7*9 = -53

6/2+8-7*9 = -52

4/1+8-7*9 = -51

5/1+8-7*9 = -50

6/1+8-7*9 = -49

2/1+4-6*9 = -48

8/4+5-6*9 = -47

3/1+5-6*9 = -46

6/3-7*8+9 = -45

6/2-7*8+9 = -44

4/1-7*8+9 = -43

5/1-7*8+9 = -42

6/1-7*8+9 = -41

3+8/4-5*9 = -40

7/1+8-6*9 = -39

6/2+4-5*9 = -38

4/2-6*8+9 = -37

3/1-6*8+9 = -36

4/1-6*8+9 = -35

5/1-6*8+9 = -34

4/1+8-5*9 = -33

7/1-6*8+9 = -32

8/4-6*7+9 = -31

3/1-6*7+9 = -30

6/3-5*8+9 = -29

6/2-5*8+9 = -28

4/1-5*8+9 = -27

6/3+8-4*9 = -26

6/1-5*8+9 = -25

7/1-5*8+9 = -24

6/2-5*7+9 = -23

8/2-5*7+9 = -22

6/3-4*8+9 = -21

6/2-4*8+9 = -20

8/4-5*6+9 = -19

5/1-4*8+9 = -18

6/1-4*8+9 = -17

7/1-4*8+9 = -16

4-7/2*8+9 = -15

5-7/2*8+9 = -14

4/2-3*8+9 = -13

6/2-3*8+9 = -12


Por tanto, hemos diseñado un algoritmo por fuerza bruta que calcula todas las permutaciones posibles y las almacena en un diccionario para posteriormente buscar en este diccionario la solución buscada. Este método tiene varias ventajas e inconvenientes. 

Ventajas:

* Sólo es necesario correr el código una vez para cada combinación de operaciones y cifras.

* Para conjuntos de datos no demasiado elevados, el coste computacional no es demasiado elevado.

Inconvenientes:

* Almacena una única solución para cada valor posible, deshechando la mayoría de combinaciones calculadas.

* El coste computacional escala con el cuadrado de un factorial, es decir, demasiado rápido.


----

**Calcula la complejidad del algoritmo por fuerza bruta**

De la documentación de la librería `itertools` obtenemos el código de la función `permutations` utilizada:

```
def permutations(iterable, r=None):
    # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    if r > n:
        return
    indices = list(range(n))
    cycles = list(range(n, n-r, -1))
    yield tuple(pool[i] for i in indices[:r])
    while n:
        for i in reversed(range(r)):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in indices[:r])
                break
        else:
            return
```
En base a este código, podemos determinar que el algorítmo necesita un mínimo de $n!$ operaciones, siendo el peor de los casos $n*n!≈n!$, por lo que la complejidad computacional de esta función es de $O(n!)$, considerando $n$ como el número total de elementos de partida (cifras + operadores), que en nuestro caso será $n=9+4=13$.

En cuanto al código para combinar ambas listas de permutaciones, en el peor de los casos tendrá una complejidad computacional de $n!*n!$ ya que es necesario recorrer ambas listas de permutaciones, cada una con un orden de $n!$ elementos. Esto nos da una complejidad computacional de $O((n!)^2)$.

El resto de operaciones, como la búsqueda del mínimo o de los valores enteros entre las soluciones, tienen complejidades despreciables en comparación con $O((n!)^2)$ por lo que esa será la complejidad computacional total del algoritmo.



-----

**Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

El algorítmo que se propone como mejora a la fuerza bruta es un algorítmo de ramificación y poda. Estos algorítmos tienen, en general, una complejidad de $O(\alpha^n)$ siendo $\alpha$ el promedio de ramificaciones por nodo, es decir, una constante. Por tanto la complejidad del algorítmo será en el peor de los casos exponencial y no factorial como en el caso de la fuerza bruta.

In [2]:
import random
from random import sample

# random.seed(73)

cifras = [str(x) for x in range(1, 10)]
operadores = ['+', '-', '/', '*']


def max_val(cifras=[]):
    # Función de valor máximo para cuatro operadores y cifras consecutivas empezando en el 1
    cifras = [int(x) for x in cifras]
    return int(cifras[-1]*cifras[-2]+cifras[-3]-cifras[1]/cifras[0])


def min_val(cifras=[]):
    # Función de valor mínimo para cuatro operadores y cifras consecutivas empezando en el 1
    cifras = [int(x) for x in cifras]
    return int(cifras[0]+cifras[3]/cifras[1]-cifras[-1]*cifras[-2])


def generardor_nodos(padre=None, cifras=[], operadores=[]):
    # Añade una cifra y una operación aleatorias al nodo dado. O(1)

    if padre is None:
        return sample(operadores, 1)[0].join(sample(cifras, 2))
    else:
        ops_left = list(set(operadores) - set(padre) - set(cifras))
        num_left = list(set(cifras) - set(padre) - set(operadores))

        if len(ops_left) >= 1:
            return padre + sample(ops_left, 1)[0] + (sample(num_left, 1))[0]
        else:
            return padre


def cotas(nodo, cifras=[]):
    # Calcula los límites superior e inferior de un nodo a partir de las cifras y operaciones restantes.
    # Para ello se han elegido operadores que en principio maximizan o minimizan las operaciones. O(1)

    if eval(nodo) > 0:
        ops_max = ["*", "+"]
        ops_min = ["/", "-"]
    else:
        ops_max = ["/", "+"]
        ops_min = ["*", "-"]

    num_left = list(set(cifras)-set(nodo))
    ops_max_left = list(set(ops_max) - set(nodo))
    ops_min_left = list(set(ops_min) - set(nodo))

    if len(ops_max_left) > 0:
        max_res = eval(nodo + ops_max_left[0] + max(num_left))
    else:
        max_res = eval(nodo)

    if len(ops_min_left) > 0:
        min_res = eval(nodo + ops_min_left[0] + max(num_left))
    else:
        min_res = eval(nodo)

    return max_res, min_res


def evaluar_nodo(nodo, descartes, objetivo, cifras =[]):
    """Esta función es la responsable de la poda. Se añade a la lista de descartes todo aquel nodo que:
        1. Habiendo empleado la división y el producto no de un resultado entero.
        2. Se aleje más allá de las cotas máximas y mínimas estimadas para las cifras y operadores restantes."""
    # O(1)
    cota_max, cota_min = cotas(nodo, cifras)

    if '/' in nodo and '*' in nodo:
        if not eval(nodo).is_integer():
            descartes.append(nodo)
            return False, descartes

    if cota_max > objetivo and cota_min < objetivo:
        return True, descartes
    else:
        descartes.append(nodo)
        return False, descartes


def ryp(objetivo, nodo=None, descartes=[], cifras=[], operadores=[], count=0):
    # Función de control recursiva. Es la función que en última instancia devuelve el resultado. O(2^n) (caso típico)
    if count == 0:
        # Nos aseguramos de que el objetivo esté entre los valores posibles de obtener
        if objetivo not in range(min_val(cifras), max_val(cifras)+1):
            print('El resultado no puede obtenerse')
            return '0'

    if count > 7:
        # En este punto encontrar el resultado es difcil. Reiniciamos el nodo.
        return ryp(objetivo, None, descartes, cifras, operadores, 0)

    # Nueva generación del nodo a partir del anterior. La generación es aleatoria.
    nodo_new = generardor_nodos(nodo, cifras, operadores)

    # Si el nuevo nodo ya ha sido descartado, buscamos otro.
    if nodo_new in descartes:
        count += 1
        return ryp(objetivo, nodo, descartes, cifras, operadores, count)

    # Si la expresión está completa, se evalua. En caso de no satisfacer la evaluación, se itera de nuevo.
    if len(nodo_new) == 9:
        if eval(nodo_new) == objetivo:
            return nodo_new
        else:
            descartes.append(nodo_new)
            return ryp(objetivo, nodo, descartes, cifras, operadores, count)

    # Si la expresión no está completa se evalua el nodo y se decide si continuar o iterar de nuevo.
    continuar, descartes = evaluar_nodo(nodo_new, descartes, objetivo, cifras)

    if continuar:
        return ryp(objetivo, nodo_new, descartes, cifras, operadores, count)
    else:
        return ryp(objetivo, nodo, descartes, cifras, operadores, count)

res = ryp(-6, None, [], cifras, operadores, 0)
print(f'La expresión encontrada es: {res} = {int(eval(res))}')


La expresión encontrada es: 1-6/2*4+5 = -6


Visto el funcionamiento del algorimo, se puede concluir que efectivamente mejora con creces el algoritmo de fuerza bruta diseñado en cuanto a complejidad computacional. Por otra parte, tal y como se ha mencionado anteriormente, el algorítmo por fuerza bruta diseñado tiene la ventaja de ofrecer todos los posibles resultados, mientras que éste algoritmo sólo nos permite encontrar uno cada vez.

------

**Calcula la complejidad del algoritmo** 

Cómo se ha mencionado en la sección anterior, los algorítmos de ramificación y poda tienen una complejidad computacional de $O(\alpha^n)$ siendo $\alpha$ el promedio de ramificaciones por nodo. En este caso, el resto de funciones a las que se llama en la función recursiva no aportan mayor complejidad, por lo que podemos decir que la complejidad del algorítmo es exponencial. Cabe destacar, que al ser un algorítmo que genera nodos de forma aleatoria, hay cierta variabilidad en cuanto al tiempo de ejecución.

----------------------

**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**

Para el problema que nos ocupa, modificar los operadores requería modificar también el algorítmo, sin embargo, en cuanto a las cifras la única condición es que sean enteras, consecutivas, y que empiecen en 1. En este sentido, podríamos diseñar un juego de datos nuevo con una mayor cantidad de cifras y comprobar si el algorítmo es capaz de resolverlo.

In [12]:
cifras = [str(x) for x in range(1, 7)]
operadores = ['+', '-', '/', '*']

res = ryp(15, cifras=cifras, operadores=operadores)
print(f'La expresión encontrada es: {res} = {int(eval(res))}')

La expresión encontrada es: 6*3/1-5+2 = 15


En general, el algorítmo no funciona para sets de números mayores a 9, ya que la longitud del string resultante es mayor de lo esperado, pero sí funciona para sets de números entre 5 y 9.

------------

**Referencias**

En general, se han complementado los apuntes de clase con búsquedas en la web. Tanto en foros como en la documentación de las distintas librerias empleadas para realizar el trabajo.

----------

**Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

Avanzar en el estudio del problema implicaría generalizar y aportar robustez al algoritmo. Hay varias fallas que el algoritmo diseñado muestra y que no han podido corregirse, como por ejemplo errores inesperados debido a la naturaleza aleatoria del mismo, o las propias limitaciones encontradas cuando se ha tratado de generalizar.

Otro punto interesante sería añadir operaciones al mismo, como pueden ser las potencias, aunque es posible que esto eliminase la garantía de obtener todos los enteros posibles entre un máximo y un mínimo.